In [1]:
print("hello")

hello


In [1]:
from sourcing_selenium_BeautifulSoup import nasdaq_corporate_actions


In [2]:
k=nasdaq_corporate_actions()

Data saved to nasdaq_corporate_actions_with_Article_content.csv
           Date                                           Headline  \
0  Feb 05, 2025  Equity Corporate Actions Alert #2025-49 - Info...   

                                                Link  \
0  https://www.nasdaqtrader.com/TraderNews.aspx?i...   

                                     Article_Content  
0  The tender offer by Crown Laboratories, Inc. t...  


c:\Users\hackathon\DTCC_Sourcing\sourcing_selenium_BeautifulSoup.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Article_Content"] = df["Link"].apply(lambda url: scrape_article_content(url) if pd.notna(url) and url.startswith("http") else "")


In [5]:
k.columns

Index(['Date', 'Headline', 'Link', 'Article_Content'], dtype='object')

In [16]:
import pandas as pd
from datetime import datetime
from anthropic_client import get_anthropic_client
from prompt_templates import PROMPT_TEMPLATES, corporate_action_prompt
import json
# Initialize the AnthropicBedrock client
client = get_anthropic_client()

def invoke_claude(prompt):
    """Sends a prompt to Claude 3.5 on AWS Bedrock and returns the response."""
    try:
        response = client.messages.create(
            model="anthropic.claude-3-5-sonnet-20241022-v2:0",
            max_tokens=200,
            temperature=0.5,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content
    except Exception as e:
        print(f"ERROR: Can't invoke the model. Reason: {e}")
        return None

def process_corporate_actions_from_df(df):
    """Process extracted text to determine corporate actions and extract details from dataframe."""
    formatted_data = []
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    for index, row in df.iterrows():
        # Determine corporate action type from the Headline
        headline = row['Article_Content']
        prompt = corporate_action_prompt.format(data=headline)
        corporate_action = invoke_claude(prompt)
        corporate_action=corporate_action[0].text
        print(corporate_action)
        # Assuming that we have valid corporate action, process further
        if corporate_action and corporate_action != "No Corporate Action Found":
            corporate_action = corporate_action.strip()
            
            # Extract details based on corporate action type from the entire dictionary (row)
            detail_prompt = PROMPT_TEMPLATES[corporate_action].format(data=row.to_dict())
            detail_response = invoke_claude(detail_prompt)
        
            if detail_response:
                # Extract Company Name and Announcement Date (example logic, adjust as needed)
                extracted_info = detail_response[0].text.split("\n")
                company_name = next((line.split(": ")[1] for line in extracted_info if "Company_Name:" in line), "Unknown")
                announcement_date = next((line.split(": ")[1] for line in extracted_info if "Announcement_Date:" in line), datetime.now().strftime("%Y-%m-%d"))
                
                formatted_data.append({
                    "Company": company_name,
                    "Corporate_Action": corporate_action,
                    "Date_Announcement": announcement_date,
                    "Source": row['Link'],
                    "Headline": row['Headline'],
                    "Extracted_Information": detail_response[0].text,
                    "Insertion_Date_Time": current_datetime,
                    "Modified_Date_Time": current_datetime
                })
    
    return formatted_data

# Assuming df is your input dataframe
# df = pd.read_csv('your_dataframe.csv')  # Example loading




In [17]:
k

,Date,Headline,Link,Article_Content
0,"Feb 05, 2025",Equity Corporate Actions Alert #2025-49 - Info...,https://www.nasdaqtrader.com/TraderNews.aspx?i...,"The tender offer by Crown Laboratories, Inc. t..."


In [18]:
formatted_result = process_corporate_actions_from_df(k)

Mergers


In [1]:
from Nasdaq_Trader_Source import process_corporate_actions

data = process_corporate_actions()
print(data)


Data saved to nasdaq_corporate_actions_with_Article_content.csv
Empty DataFrame
Columns: [Date, Headline, Link, Article_Content]
Index: []
[]


c:\Users\hackathon\DTCC_Sourcing\sourcing_selenium_BeautifulSoup.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Article_Content"] = df["Link"].apply(lambda url: scrape_article_content(url) if pd.notna(url) and url.startswith("http") else "")


In [3]:
data

[{'Company': 'Revance Therapeutics, Inc.',
  'Corporate_Action': 'Mergers',
  'Date_Announcement': '2025-02-05',
  'Source': 'https://www.nasdaqtrader.com/TraderNews.aspx?id=ECA2025-49',
  'Headline': 'Equity Corporate Actions Alert #2025-49 - Information Regarding the Tender Offer of Revance Therapeutics, Inc. (RVNC)',
  'Extracted_Information': 'Company_Name: Revance Therapeutics, Inc.\nCompany_Symbol: RVNC\nValue_Date: Feb 05, 2025\nCustodian_ID: \nAccount_Number: \nSettlement_Amount: 3.65\nCurrency: USD\nTax_Details: \nEligibility: \nDebit_Security_Details: 761330109\nRemarks: Tender offer by Crown Laboratories, Inc. to acquire all outstanding common stock\nEx-Date: \nCredit_New_Security_Asset_ID: \nStock_Merger_Ratio: \nDividend_Ratio:',
  'Insertion_Date_Time': '2025-02-05 16:41:23',
  'Modified_Date_Time': '2025-02-05 16:41:23'}]

In [4]:
import requests

# Define the API endpoint
url = "http://74.249.184.110:8000/insert-corporate-action/"

# Select the first 3 elements
first_three_records = data

# Loop through the first three elements and send POST requests
for record in first_three_records:
    response = requests.post(url, json=record)
    print(f"Status Code: {response.status_code}, Response: {response.json()}")

Status Code: 200, Response: {'message': "New record inserted with status 'Not Verified'"}


In [2]:
from FINRA_Source import get_formatted_financial_data

In [3]:
extracted_data = get_formatted_financial_data()

EmptyDataError: No columns to parse from file